In [2]:
import requests
import pandas as pd

In [3]:
url = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey=O0TC2BS57NTIS0AF'
data = pd.read_csv(url)
stock_data = data[(data['assetType'] == 'Stock') & (data['exchange'] == 'NASDAQ')]
nasdaq_symbols = stock_data['symbol'].values

In [23]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&interval=5min&outputsize=full&apikey=O0TC2BS57NTIS0AF'
r = requests.get(url)
data = r.json()

df = pd.DataFrame(data['Time Series (Daily)']).T
df.columns = ['open', 'high', 'low', 'close', 'volume']
df = df.reset_index().rename(columns={'index': 'date'})
df['date'] = pd.to_datetime(df['date'])
print(df)

           date      open      high       low     close    volume
0    2024-03-06  193.5000  198.1300  192.9600  196.1600   6945818
1    2024-03-05  192.0000  193.9400  190.5700  191.9500   5653641
2    2024-03-04  187.7600  193.8980  187.6000  193.0600   7938266
3    2024-03-01  185.4900  188.3800  185.1800  188.2000   4018354
4    2024-02-29  186.1500  186.8495  184.6900  185.0300   6458487
...         ...       ...       ...       ...       ...       ...
6120 1999-11-05   92.7500   92.9400   90.1900   90.2500  13737600
6121 1999-11-04   94.4400   94.4400   90.0000   91.5600  16697600
6122 1999-11-03   95.8700   95.9400   93.5000   94.3700  10369100
6123 1999-11-02   96.7500   96.8100   93.6900   94.8100  11105400
6124 1999-11-01   98.5000   98.8100   96.3700   96.7500   9551800

[6125 rows x 6 columns]


In [5]:
url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=IBM&interval=5min&apikey=O0TC2BS57NTIS0AF'
r = requests.get(url)
data = r.json()

print(data)

{'Symbol': 'IBM', 'AssetType': 'Common Stock', 'Name': 'International Business Machines', 'Description': 'International Business Machines Corporation (IBM) is an American multinational technology company headquartered in Armonk, New York, with operations in over 170 countries. The company began in 1911, founded in Endicott, New York, as the Computing-Tabulating-Recording Company (CTR) and was renamed International Business Machines in 1924. IBM is incorporated in New York. IBM produces and sells computer hardware, middleware and software, and provides hosting and consulting services in areas ranging from mainframe computers to nanotechnology. IBM is also a major research organization, holding the record for most annual U.S. patents generated by a business (as of 2020) for 28 consecutive years. Inventions by IBM include the automated teller machine (ATM), the floppy disk, the hard disk drive, the magnetic stripe card, the relational database, the SQL programming language, the UPC barcod

In [28]:
url = 'https://www.alphavantage.co/query?function=RSI&symbol=IBM&interval=daily&time_period=100&series_type=open&apikey=O0TC2BS57NTIS0AF'
r = requests.get(url)
data = r.json()
rsi_df = pd.DataFrame(data['Technical Analysis: RSI']).T
rsi_df = rsi_df.reset_index().rename(columns={'index': 'date'})
rsi_df.index = pd.to_datetime(rsi_df.index)
df['RSI'] = rsi_df['RSI']
print(df)

KeyError: 'Technical Analysis: RSI'

In [15]:
url = 'https://www.alphavantage.co/query?function=BBANDS&symbol=IBM&interval=daily&time_period=5&series_type=close&nbdevup=3&nbdevdn=3&apikey=O0TC2BS57NTIS0AF'
r = requests.get(url)
data = r.json()
# df = pd.DataFrame(data['Technical Analysis: BBANDS']).T
# df.columns = ['Real Upper Band','Real Middle Band', 'Real Lower Band']


df['Real Upper Band'] = pd.DataFrame(data['Technical Analysis: BBANDS']).T['Real Upper Band']
df['Real Middle Band'] = pd.DataFrame(data['Technical Analysis: BBANDS']).T['Real Middle Band']
df['Real Lower Band'] = pd.DataFrame(data['Technical Analysis: BBANDS']).T['Real Lower Band']


# df = df.reset_index().rename(columns={'index': 'date'})
print(df)

            date Real Upper Band Real Middle Band Real Lower Band
0     2024-03-06        202.5129         190.8800        179.2471
1     2024-03-05        198.6444         188.7080        178.7716
2     2024-03-04        196.6878         187.2920        177.8962
3     2024-03-01        189.7113         185.5060        181.3007
4     2024-02-29        186.5876         185.0100        183.4324
...          ...             ...              ...             ...
6116  1999-11-11         52.4215          48.9231         45.4247
6117  1999-11-10         52.2262          48.5526         44.8791
6118  1999-11-09         50.7798          48.2662         45.7525
6119  1999-11-08         51.1612          48.3775         45.5939
6120  1999-11-05         52.3068          48.6573         45.0079

[6121 rows x 4 columns]


In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mp
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from datetime import date
import pandas_ta as ta
import yfinance as yf
from yahoofinancials import YahooFinancials
from sklearn.model_selection import train_test_split

In [83]:
ticker = yf.Ticker('TATAMOTORS.NS')
data = ticker.history(period="20y")

data["SMA 20"] = ta.sma(data["Close"], length = 20)
data["RSI"]=ta.rsi(data['Close'],length = 14)
data["bandwidth_bbands"] = ta.bbands(data["Close"])["BBB_5_2.0"]
data["percentB_bbands"] = ta.bbands(data["Close"])["BBP_5_2.0"]
data["MACD_12_26_9"]= ta.macd(data["Close"])["MACD_12_26_9"]
data.dropna(inplace=True)
data = data.reindex(columns=['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'SMA 20', 'RSI', 'bandwidth_bbands', 'percentB_bbands', 'MACD_12_26_9', 'Close'])
train, test = train_test_split(data, test_size=0.2, shuffle=False)
train.to_csv('dataset/train_TATAMOTORS.csv', index=False)
test.to_csv('dataset/test_TATAMOTORS.csv', index=False)

KeyboardInterrupt: 

In [72]:
ticker = yf.Ticker('RELIANCE.NS')
data = ticker.history(period="20y")

data["SMA 20"] = ta.sma(data["Close"], length = 20)
data["RSI"]=ta.rsi(data['Close'],length = 14)
data["bandwidth_bbands"] = ta.bbands(data["Close"])["BBB_5_2.0"]
data["percentB_bbands"] = ta.bbands(data["Close"])["BBP_5_2.0"]
data["MACD_12_26_9"]= ta.macd(data["Close"])["MACD_12_26_9"]
data.dropna(inplace=True)
data = data.reindex(columns=['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'SMA 20', 'RSI', 'bandwidth_bbands', 'percentB_bbands', 'MACD_12_26_9', 'Close'])
train, test = train_test_split(data, test_size=0.2, shuffle=False)
train.to_csv('dataset/train_RELIANCE.csv', index=False)
test.to_csv('dataset/test_RELIANCE.csv', index=False)

In [79]:
ticker = yf.Ticker('DIVISLAB.NS')
data = ticker.history(period="20y")
print(data)
data["SMA 20"] = ta.sma(data["Close"], length = 20)
data["RSI"]=ta.rsi(data['Close'],length = 14)
data["bandwidth_bbands"] = ta.bbands(data["Close"])["BBB_5_2.0"]
data["percentB_bbands"] = ta.bbands(data["Close"])["BBP_5_2.0"]
data["MACD_12_26_9"]= ta.macd(data["Close"])["MACD_12_26_9"]
data.dropna(inplace=True)
data = data.reindex(columns=['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'SMA 20', 'RSI', 'bandwidth_bbands', 'percentB_bbands', 'MACD_12_26_9', 'Close'])
train, test = train_test_split(data, test_size=0.2, shuffle=False)
train.to_csv('dataset/train_DIVISLAB.csv', index=False)
test.to_csv('dataset/test_DIVISLAB.csv', index=False)

                                  Open         High          Low        Close  \
Date                                                                            
2004-03-08 00:00:00+05:30    67.535457    75.374039    67.535457    75.006805   
2004-03-09 00:00:00+05:30    73.658145    77.963662    73.111513    73.852318   
2004-03-10 00:00:00+05:30    73.869180    74.924454    71.336523    72.142754   
2004-03-11 00:00:00+05:30    71.632014    73.803770    70.703374    71.093826   
2004-03-12 00:00:00+05:30    71.252108    72.094215    69.648090    70.722359   
...                                ...          ...          ...          ...   
2024-03-01 00:00:00+05:30  3491.000000  3519.000000  3474.000000  3482.399902   
2024-03-04 00:00:00+05:30  3514.800049  3542.000000  3478.000000  3482.300049   
2024-03-05 00:00:00+05:30  3484.600098  3496.500000  3452.000000  3484.449951   
2024-03-06 00:00:00+05:30  3485.000000  3560.000000  3450.000000  3544.649902   
2024-03-07 00:00:00+05:30  3

In [84]:
ticker = yf.Ticker('TATAMOTORS.NS')
data = ticker.history(period="20y")

data["SMA 20"] = ta.sma(data["Close"], length = 20)
data["RSI"]=ta.rsi(data['Close'],length = 14)
data["bandwidth_bbands"] = ta.bbands(data["Close"])["BBB_5_2.0"]
data["percentB_bbands"] = ta.bbands(data["Close"])["BBP_5_2.0"]
data["MACD_12_26_9"]= ta.macd(data["Close"])["MACD_12_26_9"]
data.dropna(inplace=True)
data = data.reindex(columns=['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'SMA 20', 'RSI', 'bandwidth_bbands', 'percentB_bbands', 'MACD_12_26_9', 'Close'])
train, test = train_test_split(data, test_size=0.2, shuffle=False)
train.to_csv('dataset/train_TATAMOTORS.csv', index=False)
test.to_csv('dataset/test_TATAMOTORS.csv', index=False)

In [85]:
data

,Open,High,Low,Volume,Dividends,Stock Splits,SMA 20,RSI,bandwidth_bbands,percentB_bbands,MACD_12_26_9,Close
Date,,,,,,,,,,,,
2004-04-12 00:00:00+05:30,80.220610,81.211478,78.555958,11237354,0.0,0.0,75.199292,54.312028,1.416628,0.313822,1.948256,79.134621
2004-04-13 00:00:00+05:30,79.269378,81.393796,77.248008,19402334,0.0,0.0,75.608718,59.658630,3.504218,0.957785,2.028590,80.934036
2004-04-14 00:00:00+05:30,80.934036,80.934036,80.934036,0,0.0,0.0,75.951162,59.658630,3.694462,0.796422,2.068412,80.934036
2004-04-15 00:00:00+05:30,81.346242,82.202352,77.533383,23485482,0.0,0.0,76.411717,59.410021,3.812868,0.687438,2.071614,80.878555
2004-04-16 00:00:00+05:30,81.100503,82.123076,80.236466,17440048,0.0,0.0,76.823918,58.398966,3.450065,0.555935,2.033440,80.664520
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01 00:00:00+05:30,958.950012,980.400024,956.700012,10645369,0.0,0.0,933.519995,73.733775,5.601038,0.879440,30.643660,977.400024
2024-03-04 00:00:00+05:30,993.250000,995.000000,980.099976,6084495,0.0,0.0,936.539996,75.423400,5.539732,0.874986,31.931492,987.200012
2024-03-05 00:00:00+05:30,1027.000000,1065.599976,1005.349976,59811033,0.0,0.0,940.657498,80.264370,10.302164,0.925867,35.344678,1021.900024
